# Setup 

### Import libraries

In [1]:
import yaml 
from pathlib import Path
import os
path = Path("C:/Users/HP/Desktop/UNOS data/codes/simulation-waitline-unos").resolve()  # Get the absolute path
os.chdir(path)
import prediction as pred 
from data import *
from simulation import * 
import copy

Load data from UNOS and dictionaries to map ethnicities and blood type compatibility

In [2]:
df_base = load_data()   
yaml_file_path = "C:/Users/HP/Desktop/UNOS data/codes/simulation-waitline-unos/mapping_data.yaml"
with open(yaml_file_path, "r") as file:
    loaded_data = yaml.safe_load(file)
ethcat_mapping = loaded_data["ethcat_mapping"]
blood_type_compatibility = loaded_data["blood_type_compatibility"]

Get proportions by gender from UNOS sample

In [3]:
real_gender_df = df_base.GENDER.value_counts(normalize=True).reset_index()
real_gender_df.columns = ['gender', 'sample_proportion']
real_gender_df

,gender,sample_proportion
0,M,0.64827
1,F,0.35173


Get proportions by ethnicity from UNOS sample

In [4]:
real_ethcat_df = df_base.ETHCAT.value_counts(normalize=True).reset_index()
real_ethcat_df.ETHCAT = real_ethcat_df.ETHCAT.map(ethcat_mapping)
real_ethcat_df.columns = ['ethcat', 'sample_proportion']
real_ethcat_df

,ethcat,sample_proportion
0,"White, Non-Hispanic",0.439416
1,"Black, Non-Hispanic",0.325258
2,Hispanic/Latino,0.159551
3,"Asian, Non-Hispanic",0.053213
4,"Amer Ind/Alaska Native, Non-Hispanic",0.013618
5,"Multiracial, Non-Hispanic",0.004764
6,"Native Hawaiian/Other Pacific Islander, Non-Hi...",0.004180


Load predictors and create events

In [5]:
decision_tree = pred.SurvivalPredictionModel(model_path='models\decision_tree_1212.pkl')
cox_prop = pred.SurvivalPredictionModel(model_path='models\coxprop_0912.pkl')
random_forest = pred.SurvivalPredictionModel(model_path=r'models\forest_0912.pkl')

replicates = 5
events_per_replication =create_list_of_events(n_events = 10000, replications=replicates)

events_per_replication_p1 = copy.deepcopy(events_per_replication)
events_per_replication_p2 = copy.deepcopy(events_per_replication)
events_per_replication_p3 = copy.deepcopy(events_per_replication)



<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\c'
<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\c'
C:\Users\HP\AppData\Local\Temp\ipykernel_14392\142732818.py:1: SyntaxWarning: invalid escape sequence '\d'
  decision_tree = pred.SurvivalPredictionModel(model_path='models\decision_tree_1212.pkl')
C:\Users\HP\AppData\Local\Temp\ipykernel_14392\142732818.py:2: SyntaxWarning: invalid escape sequence '\c'
  cox_prop = pred.SurvivalPredictionModel(model_path='models\coxprop_0912.pkl')


models\decision_tree_1212.pkl
Model loaded successfully from models\decision_tree_1212.pkl.
models\coxprop_0912.pkl
Model loaded successfully from models\coxprop_0912.pkl.
models\forest_0912.pkl
Model loaded successfully from models\forest_0912.pkl.


# Survival Decision tree

## Policy 1

In [6]:
average_waiting_times, number_of_matches_list, results_p1_no, results_p1_ll, results_p1_fl = run_simulation(events_per_replication_p1,  replicates, decision_tree, policy='p1', verbose=False)

Replicate # 0


Progress replication 0: 10959.15887291045time unit [03:44, 48.73time unit/s]                        


Replicate # 1


Progress replication 1: 10958.559094472406time unit [03:34, 51.15time unit/s]                       


Replicate # 2


Progress replication 2: 10959.068290092859time unit [03:44, 48.76time unit/s]                      


Replicate # 3


Progress replication 3: 10959.846562314464time unit [03:49, 47.80time unit/s]                       


Replicate # 4


Progress replication 4: 100%|█████████▉| 10957.24190083592/10957.5 [03:49<00:00, 66.52time unit/s]  c:\Users\HP\miniconda3\Lib\site-packages\tqdm\std.py:524: TqdmWarning: clamping frac to range [0, 1]
  full_bar = Bar(frac,
Progress replication 4: 100%|██████████| 10957.58944581508/10957.5 [03:50<00:00, 47.63time unit/s]


### recipients who receive a kidney

In [7]:
df_waiting_time_no, df_gender_no = get_summary_tables(results_p1_no, ethcat_mapping, real_ethcat_df, real_gender_df)

In [8]:

df_waiting_time_no.to_csv('results/decision_tree_p1_eth_no.csv') 
df_waiting_time_no

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",1150.362,64.446,17.0,2.121,2879.100,178.798,0.013,0.014
1,"Asian, Non-Hispanic",1196.308,96.239,63.8,5.167,2751.499,142.003,0.047,0.053
2,"Black, Non-Hispanic",1200.614,95.018,431.4,20.707,2832.122,64.642,0.318,0.325
3,Hispanic/Latino,1168.831,94.646,233.6,9.317,2897.302,115.603,0.172,0.160
4,"Multiracial, Non-Hispanic",1289.041,127.518,8.8,2.168,3209.116,269.770,0.006,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",1162.004,134.288,5.4,1.140,3088.731,216.346,0.004,0.004
6,"White, Non-Hispanic",1220.622,104.075,595.8,18.553,2767.504,37.069,0.439,0.439


In [9]:

df_gender_no.to_csv('results/decision_tree_p1_gender_no.csv')
df_gender_no

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,1195.591395,94.169511,479.2,20.801442,2821.686129,56.170632,0.353444,0.35173
1,M,1207.809317,99.426629,876.6,25.481366,2810.671950,38.103677,0.646556,0.64827


### patients who leave the waitlist

In [10]:
df_waiting_time_ll, df_gender_ll = get_summary_tables(results_p1_ll, ethcat_mapping, real_ethcat_df, real_gender_df)

In [11]:
df_waiting_time_ll.to_csv('results/decision_tree_p1_eth_ll.csv')
df_waiting_time_ll

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",565.024,149.065,8.0,1.414,0.0,0.0,0.014,0.014
1,"Asian, Non-Hispanic",643.328,54.454,31.2,1.483,0.0,0.0,0.057,0.053
2,"Black, Non-Hispanic",562.781,56.310,183.4,15.291,0.0,0.0,0.332,0.325
3,Hispanic/Latino,576.635,70.670,91.4,4.278,0.0,0.0,0.166,0.160
4,"Multiracial, Non-Hispanic",536.765,196.605,3.0,1.225,0.0,0.0,0.005,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",755.486,78.274,2.8,0.447,0.0,0.0,0.005,0.004
6,"White, Non-Hispanic",589.317,42.794,232.2,13.480,0.0,0.0,0.421,0.439


In [12]:
df_gender_ll.to_csv('results/decision_tree_p1_gender_ll.csv')
df_gender_ll

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,571.603542,41.393205,201.4,16.876018,0.0,0.0,0.364855,0.35173
1,M,588.238607,55.355716,350.6,18.796276,0.0,0.0,0.635145,0.64827


### patients who stay in final list

In [13]:
df_waiting_time_fl, df_gender_fl = get_summary_tables(results_p1_fl, ethcat_mapping, real_ethcat_df, real_gender_df)

In [14]:
df_waiting_time_fl.to_csv('results/decision_tree_p1_eth_fl.csv')
df_waiting_time_fl

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",486.718,122.452,3.0,1.225,0.0,0.0,0.012,0.014
1,"Asian, Non-Hispanic",526.158,212.330,9.6,2.191,0.0,0.0,0.039,0.053
2,"Black, Non-Hispanic",526.932,46.381,92.0,4.062,0.0,0.0,0.374,0.325
3,Hispanic/Latino,590.622,94.211,36.4,8.444,0.0,0.0,0.148,0.160
4,"Multiracial, Non-Hispanic",142.680,96.346,1.5,0.577,0.0,0.0,0.006,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",845.537,193.391,1.0,0.000,0.0,0.0,0.004,0.004
6,"White, Non-Hispanic",600.801,59.536,102.2,11.735,0.0,0.0,0.416,0.439


In [15]:
df_gender_fl.to_csv('results/decision_tree_p1_gender_fl.csv')
df_gender_fl

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,549.148758,66.039985,84.0,1.581139,0.0,0.0,0.342857,0.35173
1,M,576.996612,59.630424,161.0,14.177447,0.0,0.0,0.657143,0.64827


In [16]:
get_probability_by_ethinicy(df_waiting_time_no, df_waiting_time_ll, df_waiting_time_fl).to_csv('results/decision_tree_p1_eth_prob.csv')

## Policy 2

In [17]:
average_waiting_times, number_of_matches_list, results_p2_no, results_p2_ll, results_p2_fl = run_simulation(events_per_replication_p2,  replicates, decision_tree, policy='p2', verbose=False)

Replicate # 0


Progress replication 0: 10959.15887291045time unit [01:11, 154.09time unit/s]                       


Replicate # 1


Progress replication 1: 10958.559094472406time unit [01:07, 163.37time unit/s]                      


Replicate # 2


Progress replication 2: 10959.068290092859time unit [01:08, 159.13time unit/s]                      


Replicate # 3


Progress replication 3: 10959.846562314464time unit [01:10, 156.30time unit/s]                      


Replicate # 4


Progress replication 4: 100%|█████████▉| 10957.24190083592/10957.5 [01:07<00:00, 166.63time unit/s] c:\Users\HP\miniconda3\Lib\site-packages\tqdm\std.py:524: TqdmWarning: clamping frac to range [0, 1]
  full_bar = Bar(frac,
Progress replication 4: 100%|██████████| 10957.58944581508/10957.5 [01:07<00:00, 161.88time unit/s]


### recipients who receive a kidney

In [18]:
df_waiting_time_no, df_gender_no = get_summary_tables(results_p2_no, ethcat_mapping, real_ethcat_df, real_gender_df)

In [19]:
df_waiting_time_no.to_csv('results/decision_tree_p2_eth_no.csv') 
df_waiting_time_no

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",1070.620,68.366,17.4,2.302,2682.787,150.166,0.013,0.014
1,"Asian, Non-Hispanic",1184.987,92.636,61.8,4.970,2666.348,180.927,0.046,0.053
2,"Black, Non-Hispanic",1199.914,77.380,433.4,22.788,2588.497,42.863,0.320,0.325
3,Hispanic/Latino,1113.904,88.180,232.6,8.591,2624.977,61.494,0.172,0.160
4,"Multiracial, Non-Hispanic",1340.425,237.598,8.4,2.510,2688.292,391.310,0.006,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",1106.470,163.094,5.4,0.894,2410.549,525.056,0.004,0.004
6,"White, Non-Hispanic",1189.159,97.301,596.8,22.197,2552.780,46.815,0.440,0.439


In [20]:
df_gender_no.to_csv('results/decision_tree_p2_gender_no.csv')
df_gender_no

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,1182.248670,80.977587,482.2,20.179197,2563.237086,38.433202,0.355657,0.35173
1,M,1176.050625,89.045273,873.6,27.208455,2595.497608,55.529288,0.644343,0.64827


### patients who leave the waitlist

In [21]:
df_waiting_time_ll, df_gender_ll = get_summary_tables(results_p2_ll, ethcat_mapping, real_ethcat_df, real_gender_df)

In [22]:
df_waiting_time_ll.to_csv('results/decision_tree_p2_eth_ll.csv') 
df_waiting_time_ll

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",552.067,155.997,7.8,1.304,0.0,0.0,0.014,0.014
1,"Asian, Non-Hispanic",674.695,63.924,31.8,2.387,0.0,0.0,0.058,0.053
2,"Black, Non-Hispanic",595.394,74.161,182.0,16.912,0.0,0.0,0.331,0.325
3,Hispanic/Latino,599.245,69.762,90.0,1.871,0.0,0.0,0.164,0.160
4,"Multiracial, Non-Hispanic",536.765,196.605,3.0,1.225,0.0,0.0,0.005,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",719.175,253.186,2.8,1.095,0.0,0.0,0.005,0.004
6,"White, Non-Hispanic",637.037,65.215,232.4,13.649,0.0,0.0,0.423,0.439


In [23]:
df_gender_ll.to_csv('results/decision_tree_p2_gender_ll.csv')
df_gender_ll

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,593.118813,51.780793,199.2,15.172343,0.0,0.0,0.362314,0.35173
1,M,633.708384,74.930373,350.6,18.955210,0.0,0.0,0.637686,0.64827


### patients who stay in final list

In [24]:
df_waiting_time_fl, df_gender_fl = get_summary_tables(results_p2_fl, ethcat_mapping, real_ethcat_df, real_gender_df)

In [25]:
df_waiting_time_fl.to_csv('results/decision_tree_p2_eth_fl.csv')
df_waiting_time_fl

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",440.160,149.202,2.8,1.095,0.0,0.0,0.011,0.014
1,"Asian, Non-Hispanic",700.017,204.389,10.2,2.280,0.0,0.0,0.041,0.053
2,"Black, Non-Hispanic",582.860,34.477,92.0,5.339,0.0,0.0,0.371,0.325
3,Hispanic/Latino,711.286,53.176,38.4,7.162,0.0,0.0,0.155,0.160
4,"Multiracial, Non-Hispanic",142.680,96.346,1.5,0.577,0.0,0.0,0.006,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",1084.203,603.977,1.0,0.000,0.0,0.0,0.004,0.004
6,"White, Non-Hispanic",649.038,66.809,101.8,13.405,0.0,0.0,0.411,0.439


In [26]:
df_gender_fl.to_csv('results/decision_tree_p2_gender_fl.csv')
df_gender_fl

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,589.512875,65.957398,83.0,2.828427,0.0,0.0,0.336032,0.35173
1,M,655.217708,35.247594,164.0,11.224972,0.0,0.0,0.663968,0.64827


In [27]:
get_probability_by_ethinicy(df_waiting_time_no, df_waiting_time_ll, df_waiting_time_fl).to_csv('results/decision_tree_p2_eth_prob.csv')

## Policy 3

In [28]:
average_waiting_times, number_of_matches_list, results_p3_no, results_p3_ll, results_p3_fl = run_simulation(events_per_replication_p3,  replicates, random_forest, policy='p3', verbose=False)

Replicate # 0


Progress replication 0: 10959.15887291045time unit [17:35, 10.38time unit/s]                         


Replicate # 1


Progress replication 1: 10958.559094472406time unit [17:57, 10.17time unit/s]                       


Replicate # 2


Progress replication 2: 10959.068290092859time unit [16:56, 10.78time unit/s]                        


Replicate # 3


Progress replication 3: 10959.846562314464time unit [17:21, 10.52time unit/s]                        


Replicate # 4


Progress replication 4: 100%|█████████▉| 10957.24190083592/10957.5 [17:49<00:00, 15.69time unit/s]   c:\Users\HP\miniconda3\Lib\site-packages\tqdm\std.py:524: TqdmWarning: clamping frac to range [0, 1]
  full_bar = Bar(frac,
Progress replication 4: 100%|██████████| 10957.58944581508/10957.5 [17:50<00:00, 10.24time unit/s]


### recipients who receive a kidney

In [29]:
df_waiting_time_no, df_gender_no = get_summary_tables(results_p3_no, ethcat_mapping, real_ethcat_df, real_gender_df)

In [30]:
df_waiting_time_no.to_csv('results/decision_tree_p3_eth_no.csv')  
df_waiting_time_no

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",1105.364,72.189,17.8,2.049,3000.853,18.485,0.013,0.014
1,"Asian, Non-Hispanic",1178.865,84.965,62.6,4.037,2974.462,22.409,0.046,0.053
2,"Black, Non-Hispanic",1199.803,71.358,430.6,23.147,2986.489,3.963,0.318,0.325
3,Hispanic/Latino,1134.426,79.130,228.6,9.209,2981.637,8.912,0.169,0.160
4,"Multiracial, Non-Hispanic",1295.215,178.408,8.2,2.168,2947.255,50.665,0.006,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",1110.569,199.516,5.2,1.483,2987.324,71.062,0.004,0.004
6,"White, Non-Hispanic",1097.901,103.467,602.8,23.231,2966.731,5.019,0.445,0.439


In [31]:
df_gender_no.to_csv('results/decision_tree_p3_gender_no.csv')
df_gender_no

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,1133.739182,85.296366,484.2,18.130085,2977.619960,7.452314,0.357132,0.35173
1,M,1145.465420,86.109446,871.6,29.963311,2975.566258,4.356991,0.642868,0.64827


### patients who leave the waitlist

In [32]:
df_waiting_time_ll, df_gender_ll = get_summary_tables(results_p3_ll, ethcat_mapping, real_ethcat_df, real_gender_df)

In [33]:
df_waiting_time_ll.to_csv('results/decision_tree_p3_eth_ll.csv')
df_waiting_time_ll

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",508.611,108.223,7.6,1.140,0.0,0.0,0.014,0.014
1,"Asian, Non-Hispanic",675.349,56.611,31.0,3.082,0.0,0.0,0.057,0.053
2,"Black, Non-Hispanic",616.104,65.303,185.8,17.268,0.0,0.0,0.340,0.325
3,Hispanic/Latino,629.006,72.977,93.0,1.732,0.0,0.0,0.170,0.160
4,"Multiracial, Non-Hispanic",536.765,196.605,3.0,1.225,0.0,0.0,0.005,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",765.749,101.014,3.0,0.707,0.0,0.0,0.005,0.004
6,"White, Non-Hispanic",635.745,67.924,223.0,15.890,0.0,0.0,0.408,0.439


In [34]:
df_gender_ll.to_csv('results/decision_tree_p3_gender_ll.csv')
df_gender_ll

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,595.315471,30.256687,196.4,14.328294,0.0,0.0,0.359444,0.35173
1,M,648.641590,82.739181,350.0,22.293497,0.0,0.0,0.640556,0.64827


### patients who stay in final list

In [35]:
df_waiting_time_fl, df_gender_fl = get_summary_tables(results_p3_fl, ethcat_mapping, real_ethcat_df, real_gender_df)

In [36]:
df_waiting_time_fl.to_csv('results/decision_tree_p3_eth_fl.csv')
df_waiting_time_fl

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",440.160,149.202,2.8,1.095,0.0,0.0,0.011,0.014
1,"Asian, Non-Hispanic",725.716,195.678,10.4,1.517,0.0,0.0,0.042,0.053
2,"Black, Non-Hispanic",600.772,56.525,93.8,4.970,0.0,0.0,0.381,0.325
3,Hispanic/Latino,721.552,58.512,38.2,6.058,0.0,0.0,0.155,0.160
4,"Multiracial, Non-Hispanic",142.680,96.346,1.5,0.577,0.0,0.0,0.006,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",1084.203,603.977,1.0,0.000,0.0,0.0,0.004,0.004
6,"White, Non-Hispanic",660.939,64.902,98.2,12.578,0.0,0.0,0.399,0.439


In [37]:
df_gender_fl.to_csv('results/decision_tree_p3_gender_fl.csv')
df_gender_fl

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,605.356874,58.951362,81.2,2.049390,0.0,0.0,0.331158,0.35173
1,M,666.315961,46.181125,164.0,12.389512,0.0,0.0,0.668842,0.64827


In [38]:
get_probability_by_ethinicy(df_waiting_time_no, df_waiting_time_ll, df_waiting_time_fl).to_csv('results/decision_tree_p3_eth_prob.csv')
get_probability_by_ethinicy

<function simulation.get_probability_by_ethinicy(df_waiting_time_no, df_waiting_time_ll, df_waiting_time_fl)>

# Random survival forest

In [39]:
events_per_replication_p1 = copy.deepcopy(events_per_replication)
events_per_replication_p2 = copy.deepcopy(events_per_replication)
events_per_replication_p3 = copy.deepcopy(events_per_replication)

## Policy 1

In [40]:
average_waiting_times, number_of_matches_list, results_p1_no, results_p1_ll, results_p1_fl = run_simulation(events_per_replication_p1,  
                                                                                                            replicates, random_forest, 
                                                                                                            policy='p1', verbose=False)

Replicate # 0


Progress replication 0: 10959.15887291045time unit [09:25, 19.37time unit/s]                       


Replicate # 1


Progress replication 1: 10958.559094472406time unit [09:52, 18.49time unit/s]                      


Replicate # 2


Progress replication 2: 10959.068290092859time unit [09:33, 19.11time unit/s]                        


Replicate # 3


Progress replication 3: 10959.846562314464time unit [09:28, 19.28time unit/s]                      


Replicate # 4


Progress replication 4: 100%|█████████▉| 10957.24190083592/10957.5 [09:42<00:00, 27.22time unit/s] c:\Users\HP\miniconda3\Lib\site-packages\tqdm\std.py:524: TqdmWarning: clamping frac to range [0, 1]
  full_bar = Bar(frac,
Progress replication 4: 100%|██████████| 10957.58944581508/10957.5 [09:42<00:00, 18.80time unit/s]


### recipients who receive a kidney

In [41]:
df_waiting_time_no, df_gender_no = get_summary_tables(results_p1_no, ethcat_mapping, real_ethcat_df, real_gender_df)

In [42]:
df_waiting_time_no.to_csv('results/random_forest_p1_eth_no.csv') 
df_waiting_time_no

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",649.146,272.585,15.0,1.414,2942.665,29.013,0.011,0.014
1,"Asian, Non-Hispanic",736.131,67.783,65.2,3.834,2933.810,8.169,0.048,0.053
2,"Black, Non-Hispanic",684.636,43.280,424.4,24.735,2968.605,3.836,0.313,0.325
3,Hispanic/Latino,788.965,92.440,202.6,13.667,2966.696,11.648,0.149,0.160
4,"Multiracial, Non-Hispanic",575.034,400.819,6.6,0.894,2968.941,51.084,0.005,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",1528.154,267.248,6.2,0.837,2938.333,83.909,0.005,0.004
6,"White, Non-Hispanic",611.117,31.062,635.8,22.084,2983.025,6.131,0.469,0.439


In [43]:
df_gender_no.to_csv('results/random_forest_p1_gender_no.csv')
df_gender_no

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,647.518437,14.528119,492.8,24.190907,2973.71416,7.723878,0.363475,0.35173
1,M,685.382802,42.143304,863.0,19.659603,2972.65830,8.293637,0.636525,0.64827


### patients who leave the waitlist

In [44]:
df_waiting_time_ll, df_gender_ll = get_summary_tables(results_p1_ll, ethcat_mapping, real_ethcat_df, real_gender_df)

In [45]:
df_waiting_time_ll.to_csv('results/random_forest_p1_eth_ll.csv')
df_waiting_time_ll

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",1133.462,144.443,9.8,1.924,0.0,0.0,0.018,0.014
1,"Asian, Non-Hispanic",1637.790,173.337,30.4,5.320,0.0,0.0,0.056,0.053
2,"Black, Non-Hispanic",1590.583,100.561,190.0,11.769,0.0,0.0,0.348,0.325
3,Hispanic/Latino,1542.059,120.270,117.8,5.357,0.0,0.0,0.216,0.160
4,"Multiracial, Non-Hispanic",1877.570,596.798,3.0,1.000,0.0,0.0,0.005,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",1397.970,763.889,3.2,1.304,0.0,0.0,0.006,0.004
6,"White, Non-Hispanic",1706.213,58.500,192.4,16.395,0.0,0.0,0.352,0.439


In [46]:
df_gender_ll.to_csv('results/random_forest_p1_gender_ll.csv')
df_gender_ll

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,1577.050273,68.895908,193.6,15.436969,0.0,0.0,0.35419,0.35173
1,M,1641.055448,118.843764,353.0,20.808652,0.0,0.0,0.64581,0.64827


### patients who stay in final list

In [47]:
df_waiting_time_fl, df_gender_fl = get_summary_tables(results_p1_fl, ethcat_mapping, real_ethcat_df, real_gender_df)

In [48]:
df_waiting_time_fl.to_csv('results/random_forest_p1_eth_fl.csv')
df_waiting_time_fl

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",1183.145,392.927,4.2,2.168,0.0,0.0,0.017,0.014
1,"Asian, Non-Hispanic",2451.671,712.570,12.2,1.924,0.0,0.0,0.049,0.053
2,"Black, Non-Hispanic",1660.945,168.579,94.6,5.771,0.0,0.0,0.378,0.325
3,Hispanic/Latino,1957.444,111.866,48.0,4.848,0.0,0.0,0.192,0.160
4,"Multiracial, Non-Hispanic",1852.394,1086.962,2.8,1.304,0.0,0.0,0.011,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",1711.858,189.372,1.0,0.000,0.0,0.0,0.004,0.004
6,"White, Non-Hispanic",1871.892,244.161,87.6,9.889,0.0,0.0,0.350,0.439


In [49]:
df_gender_fl.to_csv('results/random_forest_p1_gender_fl.csv')
df_gender_fl

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,1797.399441,144.516960,78.4,4.722288,0.0,0.0,0.313349,0.35173
1,M,1846.294311,207.581795,171.8,13.386560,0.0,0.0,0.686651,0.64827


In [50]:
get_probability_by_ethinicy(df_waiting_time_no, df_waiting_time_ll, df_waiting_time_fl).to_csv('results/random_forest_p1_eth_prob.csv')

## Policy 2

In [51]:
average_waiting_times, number_of_matches_list, results_p2_no, results_p2_ll, results_p2_fl = run_simulation(events_per_replication_p2,  replicates, random_forest, policy='p2', verbose=False)

Replicate # 0


Progress replication 0: 10959.15887291045time unit [01:20, 135.71time unit/s]                       


Replicate # 1


Progress replication 1: 10958.559094472406time unit [01:18, 140.27time unit/s]                      


Replicate # 2


Progress replication 2: 10959.068290092859time unit [01:16, 143.70time unit/s]                      


Replicate # 3


Progress replication 3: 10959.846562314464time unit [01:16, 142.72time unit/s]                      


Replicate # 4


Progress replication 4: 100%|█████████▉| 10952.554555121698/10957.5 [01:13<00:00, 159.28time unit/s]c:\Users\HP\miniconda3\Lib\site-packages\tqdm\std.py:524: TqdmWarning: clamping frac to range [0, 1]
  full_bar = Bar(frac,
Progress replication 4: 100%|██████████| 10957.58944581508/10957.5 [01:13<00:00, 148.87time unit/s] 


### recipients who receive a kidney

In [52]:
df_waiting_time_no, df_gender_no = get_summary_tables(results_p2_no, ethcat_mapping, real_ethcat_df, real_gender_df)

In [53]:
df_waiting_time_no.to_csv('results/random_forest_p2_eth_no.csv')
df_waiting_time_no

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",1070.620,68.366,17.4,2.302,2956.467,38.089,0.013,0.014
1,"Asian, Non-Hispanic",1184.987,92.636,61.8,4.970,2969.624,11.361,0.046,0.053
2,"Black, Non-Hispanic",1199.914,77.380,433.4,22.788,2960.428,13.264,0.320,0.325
3,Hispanic/Latino,1113.904,88.180,232.6,8.591,2950.244,8.300,0.172,0.160
4,"Multiracial, Non-Hispanic",1340.425,237.598,8.4,2.510,3004.177,40.996,0.006,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",1106.470,163.094,5.4,0.894,2958.070,35.586,0.004,0.004
6,"White, Non-Hispanic",1189.159,97.301,596.8,22.197,2970.329,4.003,0.440,0.439


In [54]:
df_gender_no.to_csv('results/random_forest_p2_gender_no.csv')
df_gender_no

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,1182.248670,80.977587,482.2,20.179197,2964.227377,8.268057,0.355657,0.35173
1,M,1176.050625,89.045273,873.6,27.208455,2963.253861,4.574890,0.644343,0.64827


### patients who leave the waitlist

In [55]:
df_waiting_time_ll, df_gender_ll = get_summary_tables(results_p2_ll, ethcat_mapping, real_ethcat_df, real_gender_df)

In [56]:
df_waiting_time_ll.to_csv('results/random_forest_p2_eth_ll.csv')
df_waiting_time_ll

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",552.067,155.997,7.8,1.304,0.0,0.0,0.014,0.014
1,"Asian, Non-Hispanic",674.695,63.924,31.8,2.387,0.0,0.0,0.058,0.053
2,"Black, Non-Hispanic",595.394,74.161,182.0,16.912,0.0,0.0,0.331,0.325
3,Hispanic/Latino,599.245,69.762,90.0,1.871,0.0,0.0,0.164,0.160
4,"Multiracial, Non-Hispanic",536.765,196.605,3.0,1.225,0.0,0.0,0.005,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",719.175,253.186,2.8,1.095,0.0,0.0,0.005,0.004
6,"White, Non-Hispanic",637.037,65.215,232.4,13.649,0.0,0.0,0.423,0.439


In [57]:
df_gender_ll.to_csv('results/random_forest_p2_gender_ll.csv')
df_gender_ll

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,593.118813,51.780793,199.2,15.172343,0.0,0.0,0.362314,0.35173
1,M,633.708384,74.930373,350.6,18.955210,0.0,0.0,0.637686,0.64827


### patients who stay in final list

In [58]:
df_waiting_time_fl, df_gender_fl = get_summary_tables(results_p2_fl, ethcat_mapping, real_ethcat_df, real_gender_df)

In [59]:
df_waiting_time_fl.to_csv('results/random_forest_p2_eth_fl.csv')
df_waiting_time_fl

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",440.160,149.202,2.8,1.095,0.0,0.0,0.011,0.014
1,"Asian, Non-Hispanic",700.017,204.389,10.2,2.280,0.0,0.0,0.041,0.053
2,"Black, Non-Hispanic",582.860,34.477,92.0,5.339,0.0,0.0,0.371,0.325
3,Hispanic/Latino,711.286,53.176,38.4,7.162,0.0,0.0,0.155,0.160
4,"Multiracial, Non-Hispanic",142.680,96.346,1.5,0.577,0.0,0.0,0.006,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",1084.203,603.977,1.0,0.000,0.0,0.0,0.004,0.004
6,"White, Non-Hispanic",649.038,66.809,101.8,13.405,0.0,0.0,0.411,0.439


In [60]:
df_gender_fl.to_csv('results/random_forest_p2_gender_fl.csv')
df_gender_fl

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,589.512875,65.957398,83.0,2.828427,0.0,0.0,0.336032,0.35173
1,M,655.217708,35.247594,164.0,11.224972,0.0,0.0,0.663968,0.64827


In [61]:
get_probability_by_ethinicy(df_waiting_time_no, df_waiting_time_ll, df_waiting_time_fl).to_csv('results/random_forest_p2_eth_prob.csv')

## Policy 3

### recipients who receive a kidney

In [62]:
average_waiting_times, number_of_matches_list, results_p3_no, results_p3_ll, results_p3_fl = run_simulation(events_per_replication_p3,  replicates, random_forest, policy='p3', verbose=False)

Replicate # 0


Progress replication 0: 10959.15887291045time unit [17:28, 10.45time unit/s]                         


Replicate # 1


Progress replication 1: 10958.559094472406time unit [17:18, 10.55time unit/s]                      


Replicate # 2


Progress replication 2: 10959.068290092859time unit [17:21, 10.52time unit/s]                        


Replicate # 3


Progress replication 3: 10959.846562314464time unit [17:31, 10.42time unit/s]                        


Replicate # 4


Progress replication 4: 100%|█████████▉| 10957.24190083592/10957.5 [18:27<00:00, 15.77time unit/s]   c:\Users\HP\miniconda3\Lib\site-packages\tqdm\std.py:524: TqdmWarning: clamping frac to range [0, 1]
  full_bar = Bar(frac,
Progress replication 4: 100%|██████████| 10957.58944581508/10957.5 [18:28<00:00,  9.89time unit/s]


In [63]:
df_waiting_time_no, df_gender_no = get_summary_tables(results_p3_no, ethcat_mapping, real_ethcat_df, real_gender_df)

In [64]:
df_waiting_time_no.to_csv('results/random_forest_p3_eth_no.csv')
df_waiting_time_no

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",1105.364,72.189,17.8,2.049,3000.853,18.485,0.013,0.014
1,"Asian, Non-Hispanic",1178.865,84.965,62.6,4.037,2974.462,22.409,0.046,0.053
2,"Black, Non-Hispanic",1199.803,71.358,430.6,23.147,2986.489,3.963,0.318,0.325
3,Hispanic/Latino,1134.426,79.130,228.6,9.209,2981.637,8.912,0.169,0.160
4,"Multiracial, Non-Hispanic",1295.215,178.408,8.2,2.168,2947.255,50.665,0.006,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",1110.569,199.516,5.2,1.483,2987.324,71.062,0.004,0.004
6,"White, Non-Hispanic",1097.901,103.467,602.8,23.231,2966.731,5.019,0.445,0.439


In [65]:
df_gender_no.to_csv('results/random_forest_p3_gender_no.csv')
df_gender_no

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,1133.739182,85.296366,484.2,18.130085,2977.619960,7.452314,0.357132,0.35173
1,M,1145.465420,86.109446,871.6,29.963311,2975.566258,4.356991,0.642868,0.64827


### patients who leave the waitlist

In [66]:
df_waiting_time_ll, df_gender_ll = get_summary_tables(results_p3_ll, ethcat_mapping, real_ethcat_df, real_gender_df)

In [67]:
df_waiting_time_ll.to_csv('results/random_forest_p3_eth_ll.csv')
df_waiting_time_ll

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",508.611,108.223,7.6,1.140,0.0,0.0,0.014,0.014
1,"Asian, Non-Hispanic",675.349,56.611,31.0,3.082,0.0,0.0,0.057,0.053
2,"Black, Non-Hispanic",616.104,65.303,185.8,17.268,0.0,0.0,0.340,0.325
3,Hispanic/Latino,629.006,72.977,93.0,1.732,0.0,0.0,0.170,0.160
4,"Multiracial, Non-Hispanic",536.765,196.605,3.0,1.225,0.0,0.0,0.005,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",765.749,101.014,3.0,0.707,0.0,0.0,0.005,0.004
6,"White, Non-Hispanic",635.745,67.924,223.0,15.890,0.0,0.0,0.408,0.439


In [68]:
df_gender_ll.to_csv('results/random_forest_p3_gender_ll.csv')
df_gender_ll

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,595.315471,30.256687,196.4,14.328294,0.0,0.0,0.359444,0.35173
1,M,648.641590,82.739181,350.0,22.293497,0.0,0.0,0.640556,0.64827


### patients who stay in final list

In [69]:
df_waiting_time_fl, df_gender_fl = get_summary_tables(results_p3_fl, ethcat_mapping, real_ethcat_df, real_gender_df)

In [70]:
df_waiting_time_fl.to_csv('results/random_forest_p3_eth_fl.csv')
df_waiting_time_fl

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",440.160,149.202,2.8,1.095,0.0,0.0,0.011,0.014
1,"Asian, Non-Hispanic",725.716,195.678,10.4,1.517,0.0,0.0,0.042,0.053
2,"Black, Non-Hispanic",600.772,56.525,93.8,4.970,0.0,0.0,0.381,0.325
3,Hispanic/Latino,721.552,58.512,38.2,6.058,0.0,0.0,0.155,0.160
4,"Multiracial, Non-Hispanic",142.680,96.346,1.5,0.577,0.0,0.0,0.006,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",1084.203,603.977,1.0,0.000,0.0,0.0,0.004,0.004
6,"White, Non-Hispanic",660.939,64.902,98.2,12.578,0.0,0.0,0.399,0.439


In [71]:
df_gender_fl.to_csv('results/random_forest_p3_gender_fl.csv')
df_gender_fl

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,605.356874,58.951362,81.2,2.049390,0.0,0.0,0.331158,0.35173
1,M,666.315961,46.181125,164.0,12.389512,0.0,0.0,0.668842,0.64827


In [72]:
get_probability_by_ethinicy(df_waiting_time_no, df_waiting_time_ll, df_waiting_time_fl).to_csv('results/cox_prop_p1_eth_prob.csv')

# Cox proportional hazard models

In [73]:
events_per_replication_p1 = copy.deepcopy(events_per_replication)
events_per_replication_p2 = copy.deepcopy(events_per_replication)
events_per_replication_p3 = copy.deepcopy(events_per_replication)

## Policy 1

In [74]:
average_waiting_times, number_of_matches_list, results_p1_no, results_p1_ll, results_p1_fl = run_simulation(events_per_replication_p1,  
                                                                                                            replicates, cox_prop, 
                                                                                                            policy='p1', verbose=False)

Replicate # 0


Progress replication 0: 10959.15887291045time unit [03:18, 55.09time unit/s]                        


Replicate # 1


Progress replication 1: 10958.559094472406time unit [03:15, 56.15time unit/s]                       


Replicate # 2


Progress replication 2: 10959.068290092859time unit [03:16, 55.70time unit/s]                       


Replicate # 3


Progress replication 3: 10959.846562314464time unit [03:14, 56.47time unit/s]                       


Replicate # 4


Progress replication 4: 100%|█████████▉| 10949.750594461699/10957.5 [03:13<00:00, 70.64time unit/s] c:\Users\HP\miniconda3\Lib\site-packages\tqdm\std.py:524: TqdmWarning: clamping frac to range [0, 1]
  full_bar = Bar(frac,
Progress replication 4: 100%|██████████| 10957.58944581508/10957.5 [03:13<00:00, 56.50time unit/s] 


### recipients who receive a kidney

In [75]:
df_waiting_time_no, df_gender_no = get_summary_tables(results_p1_no, ethcat_mapping, real_ethcat_df, real_gender_df)

In [76]:
df_waiting_time_no.to_csv('results/cox_prop_p1_eth_no.csv')
df_waiting_time_no

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",171.702,146.868,21.0,2.236,2648.850,83.622,0.015,0.014
1,"Asian, Non-Hispanic",312.650,118.232,59.2,2.387,2592.988,54.177,0.044,0.053
2,"Black, Non-Hispanic",200.229,32.556,474.0,17.421,2679.448,15.607,0.350,0.325
3,Hispanic/Latino,304.104,55.583,227.4,12.095,2628.277,17.073,0.168,0.160
4,"Multiracial, Non-Hispanic",374.513,273.525,8.2,2.168,2620.019,165.831,0.006,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",74.836,42.666,4.2,0.837,2768.832,180.888,0.003,0.004
6,"White, Non-Hispanic",266.981,30.185,561.8,24.530,2650.663,12.963,0.414,0.439


In [77]:
df_gender_no.to_csv('results/cox_prop_p1_gender_no.csv')
df_gender_no

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,211.688016,21.960570,483.6,18.635987,2675.891661,13.529575,0.35669,0.35173
1,M,271.556251,28.269331,872.2,28.604195,2642.575713,11.256044,0.64331,0.64827


### patients who leave the waitlist

In [78]:
df_waiting_time_ll, df_gender_ll = get_summary_tables(results_p1_ll, ethcat_mapping, real_ethcat_df, real_gender_df)

In [79]:
df_waiting_time_ll.to_csv('results/cox_prop_p1_eth_ll.csv')
df_waiting_time_ll

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",1065.818,617.926,3.6,2.408,0.0,0.0,0.007,0.014
1,"Asian, Non-Hispanic",1798.701,139.739,32.8,4.025,0.0,0.0,0.060,0.053
2,"Black, Non-Hispanic",2309.924,150.476,140.2,4.494,0.0,0.0,0.256,0.325
3,Hispanic/Latino,2020.255,107.809,104.8,9.039,0.0,0.0,0.191,0.160
4,"Multiracial, Non-Hispanic",1150.452,1093.440,2.4,1.140,0.0,0.0,0.004,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",4117.456,735.502,4.6,0.894,0.0,0.0,0.008,0.004
6,"White, Non-Hispanic",2559.156,75.194,259.8,23.371,0.0,0.0,0.474,0.439


In [80]:
df_gender_ll.to_csv('results/cox_prop_p1_gender_ll.csv')
df_gender_ll

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,2368.781589,119.091264,195.2,14.481022,0.0,0.0,0.356074,0.35173
1,M,2334.574038,77.026941,353.0,28.204610,0.0,0.0,0.643926,0.64827


### patients who stay in final list

In [81]:
df_waiting_time_fl, df_gender_fl = get_summary_tables(results_p1_fl, ethcat_mapping, real_ethcat_df, real_gender_df)

In [82]:
df_waiting_time_fl.to_csv('results/cox_prop_p1_eth_fl.csv')
df_waiting_time_fl

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",2096.112,887.615,2.2,0.447,0.0,0.0,0.009,0.014
1,"Asian, Non-Hispanic",3107.686,355.964,14.6,2.074,0.0,0.0,0.060,0.053
2,"Black, Non-Hispanic",2411.397,105.855,74.4,5.128,0.0,0.0,0.304,0.325
3,Hispanic/Latino,2768.274,171.810,39.8,2.588,0.0,0.0,0.163,0.160
4,"Multiracial, Non-Hispanic",2459.744,1923.773,1.8,0.447,0.0,0.0,0.007,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",3286.789,2438.278,1.2,0.447,0.0,0.0,0.005,0.004
6,"White, Non-Hispanic",3272.265,69.584,110.6,11.194,0.0,0.0,0.452,0.439


In [83]:
df_gender_fl.to_csv('results/cox_prop_p1_gender_fl.csv')
df_gender_fl

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,3330.828667,102.434415,84.8,4.494441,0.0,0.0,0.346688,0.35173
1,M,2674.491293,135.603537,159.8,11.476062,0.0,0.0,0.653312,0.64827


In [84]:
get_probability_by_ethinicy(df_waiting_time_no, df_waiting_time_ll, df_waiting_time_fl).to_csv('results/cox_prop_p1_eth_prob.csv')

## Policy 2

In [85]:
average_waiting_times, number_of_matches_list, results_p2_no, results_p2_ll, results_p2_fl = run_simulation(events_per_replication_p2,  
                                                                                                            replicates, cox_prop, 
                                                                                                            policy='p2', verbose=False)

Replicate # 0


Progress replication 0: 10959.15887291045time unit [01:10, 154.74time unit/s]                       


Replicate # 1


Progress replication 1: 10958.559094472406time unit [01:07, 161.44time unit/s]                      


Replicate # 2


Progress replication 2: 10959.068290092859time unit [01:09, 156.69time unit/s]                      


Replicate # 3


Progress replication 3: 10959.846562314464time unit [01:10, 154.69time unit/s]                      


Replicate # 4


Progress replication 4: 100%|█████████▉| 10957.24190083592/10957.5 [01:08<00:00, 172.49time unit/s] c:\Users\HP\miniconda3\Lib\site-packages\tqdm\std.py:524: TqdmWarning: clamping frac to range [0, 1]
  full_bar = Bar(frac,
Progress replication 4: 100%|██████████| 10957.58944581508/10957.5 [01:08<00:00, 160.36time unit/s]


### recipients who receive a kidney

In [86]:
df_waiting_time_no, df_gender_no = get_summary_tables(results_p2_no, ethcat_mapping, real_ethcat_df, real_gender_df)

In [87]:
df_waiting_time_no.to_csv('results/cox_prop_p2_eth_no.csv')
df_waiting_time_no

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",1070.620,68.366,17.4,2.302,2616.661,37.190,0.013,0.014
1,"Asian, Non-Hispanic",1184.987,92.636,61.8,4.970,2530.396,23.753,0.046,0.053
2,"Black, Non-Hispanic",1199.914,77.380,433.4,22.788,2613.512,28.600,0.320,0.325
3,Hispanic/Latino,1113.904,88.180,232.6,8.591,2532.261,8.168,0.172,0.160
4,"Multiracial, Non-Hispanic",1340.425,237.598,8.4,2.510,2717.764,74.041,0.006,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",1106.470,163.094,5.4,0.894,2475.344,123.612,0.004,0.004
6,"White, Non-Hispanic",1189.159,97.301,596.8,22.197,2567.989,5.779,0.440,0.439


In [88]:
df_gender_no.to_csv('results/cox_prop_p2_gender_no.csv')
df_gender_no

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,1182.248670,80.977587,482.2,20.179197,2588.567936,20.425619,0.355657,0.35173
1,M,1176.050625,89.045273,873.6,27.208455,2568.548966,13.359664,0.644343,0.64827


### patients who leave the waitlist

In [89]:
df_waiting_time_ll, df_gender_ll = get_summary_tables(results_p2_ll, ethcat_mapping, real_ethcat_df, real_gender_df)

In [90]:
df_waiting_time_ll.to_csv('results/cox_prop_p2_eth_ll.csv')
df_waiting_time_ll

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",552.067,155.997,7.8,1.304,0.0,0.0,0.014,0.014
1,"Asian, Non-Hispanic",674.695,63.924,31.8,2.387,0.0,0.0,0.058,0.053
2,"Black, Non-Hispanic",595.394,74.161,182.0,16.912,0.0,0.0,0.331,0.325
3,Hispanic/Latino,599.245,69.762,90.0,1.871,0.0,0.0,0.164,0.160
4,"Multiracial, Non-Hispanic",536.765,196.605,3.0,1.225,0.0,0.0,0.005,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",719.175,253.186,2.8,1.095,0.0,0.0,0.005,0.004
6,"White, Non-Hispanic",637.037,65.215,232.4,13.649,0.0,0.0,0.423,0.439


In [91]:
df_gender_ll.to_csv('results/cox_prop_p2_gender_ll.csv')
df_gender_ll

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,593.118813,51.780793,199.2,15.172343,0.0,0.0,0.362314,0.35173
1,M,633.708384,74.930373,350.6,18.955210,0.0,0.0,0.637686,0.64827


### patients who stay in final list

In [92]:
df_waiting_time_fl, df_gender_fl = get_summary_tables(results_p2_fl, ethcat_mapping, real_ethcat_df, real_gender_df)

In [93]:
df_waiting_time_fl.to_csv('results/cox_prop_p2_eth_fl.csv')
df_waiting_time_fl

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",440.160,149.202,2.8,1.095,0.0,0.0,0.011,0.014
1,"Asian, Non-Hispanic",700.017,204.389,10.2,2.280,0.0,0.0,0.041,0.053
2,"Black, Non-Hispanic",582.860,34.477,92.0,5.339,0.0,0.0,0.371,0.325
3,Hispanic/Latino,711.286,53.176,38.4,7.162,0.0,0.0,0.155,0.160
4,"Multiracial, Non-Hispanic",142.680,96.346,1.5,0.577,0.0,0.0,0.006,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",1084.203,603.977,1.0,0.000,0.0,0.0,0.004,0.004
6,"White, Non-Hispanic",649.038,66.809,101.8,13.405,0.0,0.0,0.411,0.439


In [94]:
df_gender_fl.to_csv('results/cox_prop_p2_gender_fl.csv')
df_gender_fl

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,589.512875,65.957398,83.0,2.828427,0.0,0.0,0.336032,0.35173
1,M,655.217708,35.247594,164.0,11.224972,0.0,0.0,0.663968,0.64827


In [95]:
get_probability_by_ethinicy(df_waiting_time_no, df_waiting_time_ll, df_waiting_time_fl).to_csv('results/cox_prop_p2_eth_prob.csv')

## Policy 3

In [6]:
average_waiting_times, number_of_matches_list, results_p3_no, results_p3_ll, results_p3_fl = run_simulation(events_per_replication_p3,  
                                                                                                            replicates, cox_prop, 
                                                                                                            policy='p3', verbose=False)

Replicate # 0


Progress replication 0: 10959.15887291045time unit [11:14, 16.25time unit/s]                       


Replicate # 1


Progress replication 1: 10958.559094472406time unit [11:05, 16.46time unit/s]                      


Replicate # 2


Progress replication 2: 10959.068290092859time unit [10:17, 17.75time unit/s]                        


Replicate # 3


Progress replication 3: 10959.846562314464time unit [10:10, 17.94time unit/s]                      


Replicate # 4


Progress replication 4: 100%|█████████▉| 10957.24190083592/10957.5 [10:52<00:00, 24.92time unit/s] c:\Users\HP\miniconda3\Lib\site-packages\tqdm\std.py:524: TqdmWarning: clamping frac to range [0, 1]
  full_bar = Bar(frac,
Progress replication 4: 100%|██████████| 10957.58944581508/10957.5 [10:53<00:00, 16.78time unit/s]


### recipients who receive a kidney

In [7]:
df_waiting_time_no, df_gender_no = get_summary_tables(results_p3_no, ethcat_mapping, real_ethcat_df, real_gender_df)

In [8]:
df_waiting_time_no.to_csv('results/cox_prop_p3_eth_no.csv')
df_waiting_time_no

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",637.591,55.445,17.4,3.130,2650.022,60.599,0.013,0.014
1,"Asian, Non-Hispanic",1300.731,104.430,59.4,5.030,2612.293,41.538,0.044,0.053
2,"Black, Non-Hispanic",900.104,51.657,457.0,23.685,2664.628,13.895,0.337,0.325
3,Hispanic/Latino,1063.539,119.690,220.8,9.783,2615.466,28.518,0.163,0.160
4,"Multiracial, Non-Hispanic",1050.302,67.929,8.4,1.517,2561.806,92.447,0.006,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",2125.635,293.896,5.4,0.548,2625.260,169.840,0.004,0.004
6,"White, Non-Hispanic",1136.621,129.571,587.4,20.428,2625.226,6.876,0.433,0.439


In [9]:
df_gender_no.to_csv('results/cox_prop_p3_gender_no.csv')
df_gender_no

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,1020.248941,70.242143,502.6,26.082561,2638.42365,17.896980,0.370704,0.35173
1,M,1066.002265,102.353766,853.2,18.579559,2634.63137,8.563104,0.629296,0.64827


### patients who leave the waitlist

In [10]:
df_waiting_time_ll, df_gender_ll = get_summary_tables(results_p3_ll, ethcat_mapping, real_ethcat_df, real_gender_df)

In [11]:
df_waiting_time_ll.to_csv('results/cox_prop_p3_eth_ll.csv')
df_waiting_time_ll

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",738.865,207.519,7.4,2.191,0.0,0.0,0.013,0.014
1,"Asian, Non-Hispanic",1123.118,99.349,36.4,2.702,0.0,0.0,0.066,0.053
2,"Black, Non-Hispanic",944.698,15.269,153.6,13.126,0.0,0.0,0.280,0.325
3,Hispanic/Latino,1050.743,77.316,103.6,4.159,0.0,0.0,0.189,0.160
4,"Multiracial, Non-Hispanic",369.342,158.243,2.0,1.000,0.0,0.0,0.004,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",1132.037,181.852,3.4,0.548,0.0,0.0,0.006,0.004
6,"White, Non-Hispanic",1003.457,50.761,242.6,19.769,0.0,0.0,0.442,0.439


In [12]:
df_gender_ll.to_csv('results/cox_prop_p3_gender_ll.csv')
df_gender_ll

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,963.565305,33.448561,187.8,13.084342,0.0,0.0,0.342077,0.35173
1,M,1019.280341,58.836430,361.2,22.027256,0.0,0.0,0.657923,0.64827


### patients who stay in final list

In [13]:
df_waiting_time_fl, df_gender_fl = get_summary_tables(results_p3_fl, ethcat_mapping, real_ethcat_df, real_gender_df)

In [14]:
df_waiting_time_fl.to_csv('results/cox_prop_p3_eth_fl.csv')
df_waiting_time_fl

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",706.570,335.631,3.2,0.837,0.0,0.0,0.013,0.014
1,"Asian, Non-Hispanic",920.750,196.811,11.4,3.130,0.0,0.0,0.046,0.053
2,"Black, Non-Hispanic",932.424,102.678,85.6,5.595,0.0,0.0,0.345,0.325
3,Hispanic/Latino,1108.156,103.256,43.0,6.164,0.0,0.0,0.173,0.160
4,"Multiracial, Non-Hispanic",847.834,606.694,2.0,1.000,0.0,0.0,0.008,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",1417.689,413.605,1.4,0.548,0.0,0.0,0.006,0.004
6,"White, Non-Hispanic",1028.297,53.894,101.4,13.372,0.0,0.0,0.409,0.439


In [15]:
df_gender_fl.to_csv('results/cox_prop_p3_gender_fl.csv')
df_gender_fl

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,999.629496,96.926752,77.6,7.469940,0.0,0.0,0.312903,0.35173
1,M,996.048592,29.594320,170.4,8.961027,0.0,0.0,0.687097,0.64827


In [16]:
get_probability_by_ethinicy(df_waiting_time_no, df_waiting_time_ll, df_waiting_time_fl).to_csv('results/cox_prop_p3_eth_prob.csv')